In [ ]:
import pandas as pd

# from nba_api.stats.library.data import teams
# team_id_to_name = {team[0]: team[-3] for team in teams}

In [ ]:
from nba_api.stats.endpoints import *

In [ ]:
from utils import gap_manager

with gap_manager.action_gap():
    game_ids = \
        leaguegamefinder.LeagueGameFinder(league_id_nullable='00').get_data_frames()[0].drop_duplicates(subset='GAME_ID',
                                                                                                        keep='first')[
            'GAME_ID']

In [ ]:
import youtube_dl
import re

from utils import get_pbp_data, get_video_event_dict

NUMBER_OF_PLAYS = 10

game_event_action_ids = set()
video_events = list()

p = re.compile('(\s{2}|\' )([\w+ ]*)')

for _ in range(NUMBER_OF_PLAYS):
    game_id = game_ids.sample(1)
    df = get_pbp_data(game_id=game_id)
    df['DESCRIPTION'] = df['HOMEDESCRIPTION'].fillna(df['VISITORDESCRIPTION'])
    only_shots = df[df['EVENTMSGTYPE'] <= 2]
    shots_event_data = only_shots[
        ['EVENTNUM', 'EVENTMSGACTIONTYPE', 'PERIOD', 'PCTIMESTRING', 'DESCRIPTION', 'EVENTMSGTYPE',
         'VIDEO_AVAILABLE_FLAG']]

    random_shot_event_data = shots_event_data.sample(1)
    event_id, event_action_id, period, play_clock_time, description, event_msg_type, video_available_flag = random_shot_event_data.iloc[0]
    if video_available_flag:
        game_event_action_ids.add(event_action_id)
        json = get_video_event_dict(game_id=game_id, game_event_id=str(event_id))
        video_urls = json['resultSets']['Meta']['videoUrls']
        playlist = json['resultSets']['playlist']
        video_event = {'desc': playlist[0]['dsc'], 'time': play_clock_time, 'event_action_id': event_action_id,
                       'video': video_urls[0]['lurl'], }
        video_events.append(video_event)

        # if playlist[0]['dsc'] != description:
        # if playlist[0]['dsc'] != description:
        #     raise ValueError(f"{playlist[0]['dsc']} is different that {description}")
        # match = list()
        # match = p.findall(description)
        # event_msg_action = re.sub(' ', '_', match[0][1]).upper()
        print(video_event)

        game_event_id = json['parameters']['GameEventID']
        ydl_opts = {
            'output': f'{game_id}_{game_event_id}',
            '--output': f'{game_id}_{game_event_id}',
            '-o': f'{game_id}_{game_event_id}',
            'o': f'{game_id}_{game_event_id}',
        }
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_urls[0]['lurl']])

In [ ]:
print(get_video_event_dict.retry.statistics)

In [ ]:
game_event_action_ids

In [ ]:
df = pd.DataFrame(video_events)
df.value_counts('event_action_id')